**Siin on minu esimene attempt birdwatchi andmetest aru saada**

Lühidalt - birdwatchil of fail(allalaetult oli ta nimi note_request), mille eesmärk on hoida järge tweetidest, millele on palutud community noti lisamist. Plaanisin sealt võtta url-id, aga nad ei klapi olemasolevate mujal olevate tweetide infoga piisavalt, et ma saaksin tweetide veebiaadresse nii üles otsima hakata.

In [ ]:
import pandas as pd
import re

# Laeme note_request.tsv ja noteStatusHistory
df_request = pd.read_csv("data/note_request.tsv", sep="\t")
df_status = pd.read_csv("data/noteStatusHistory-00000.tsv", sep="\t")

# --- Puhasta status-tabel ---
df_status = df_status[['noteId', 'currentStatus']]
df_status['noteId'] = df_status['noteId'].astype(str)

# --- Lisa sourceLinkis olev tweetId uue veeruna ---
def extract_tweet_id_from_url(url):
    if isinstance(url, str):
        match = re.search(r"status/(\d+)", url)
        if match:
            return match.group(1)
    return None

df_request['sourceTweetId'] = df_request['sourceLink'].apply(extract_tweet_id_from_url)

# Loome lihtsa lookup-tabeli kõigi note tweetide jaoks (tuleb teiselt poolt – nt kui noteId==noteTweetId)
note_tweet_lookup = df_status.copy()
note_tweet_lookup['noteTweetId'] = note_tweet_lookup['noteId']  # oletame note tweetId = noteId (kui pole muud infot)
note_tweet_lookup = note_tweet_lookup[['noteTweetId', 'currentStatus']]

# --- Ühendame ---
df_request['sourceTweetId'] = df_request['sourceTweetId'].astype(str)
df_check = df_request.merge(note_tweet_lookup, left_on='sourceTweetId', right_on='noteTweetId', how='left')

# Näitame ainult säutsud, mille kohta leidub note
df_with_notes = df_check[~df_check['currentStatus'].isna()]

# --- Kontroll ühe säutsu kohta ---
def check_note_status_for_tweet(tweet_id):
    row = df_check[df_check['tweetId'] == tweet_id]
    if row.empty:
        return f"Tweet {tweet_id} – ei leitud üldse infot."
    elif pd.isna(row.iloc[0]['currentStatus']):
        return f"Tweet {tweet_id} – leiti link, aga note pole (või pole staatuses andmeid)."
    else:
        return f"Tweet {tweet_id} – seotud note on staatusega {row.iloc[0]['currentStatus']}."

# Kontrollime ühte säutsu
print(check_note_status_for_tweet(1893175529995460774)) 

C:\Users\molsi\AppData\Local\Temp\ipykernel_8812\816406038.py:6: DtypeWarning: Columns (10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_status = pd.read_csv("data/noteStatusHistory-00000.tsv", sep="\t")


Tweet 1893175529995460774 – leiti link, aga note pole (või pole staatuses andmeid).


**Katse, kas tweetide kätte saamine twitteri api-st on võimalik**
See muidi töötas - aga ma olen seda liiga tihti jooksutanud, seega hetkel on tulemus veidi misleading

In [ ]:
import requests
import time

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOP%2B2AEAAAAAsYeRbAqf0KTfRl52w2Kn7GR2RHs%3DAwMf7ynIdnsueFxb5BvFEJLROvzi1qowVlQ7jUq8TWQCE8rHR4'

tweet_id = "1866605307595768005"

# 1. Pärime säutsu andmed
tweet_url = f"https://api.twitter.com/2/tweets/{tweet_id}"
params = {
    "tweet.fields": "created_at,author_id,public_metrics,context_annotations",
    "expansions": "author_id"
}
headers = {
    "Authorization": f"Bearer {bearer_token}"
}

tweet_response = requests.get(tweet_url, headers=headers, params=params)

if tweet_response.status_code == 200:
    tweet_data = tweet_response.json()
    tweet_text = tweet_data.get("data", {}).get("text", "Puudub tekst")
    author_id = tweet_data.get("data", {}).get("author_id", "Tundmatu")
    
    print("Säutsu tekst:")
    print(tweet_text)
    print("Säutsu link:")
    print( tweet_data.get("data", {}).get("url", "Puudub tekst"))
    #print("tweet_data:",tweet_data)
    print(f"\nAutor ID: {author_id}")

else:
    print("Viga säutsu päringus:", tweet_response.status_code)
    exit()

# 2. Proovime vaadata, kas Community Note on lisatud (Birdwatch API public note endpoint puudub)
# Siin on ainult võimalik alternatiiv: kasutada otseteed Birdwatch web-URL-iks:
print(f"\nCommunity Notes URL (manuaalne kontroll):")
print(f"https://twitter.com/i/communitynotes/{tweet_id}")

# 3. Oota, et mitte ületada Twitter API rate limit'e
time.sleep(1)

Viga säutsu päringus: 429

Community Notes URL (manuaalne kontroll):
https://twitter.com/i/communitynotes/1866605307595768005


: 

**ALL ON JÄÄNUSED MINU KATSETEST ÜRITADA Node2Vec KÄIMA SAADA**

In [16]:
#!pip install node2vec
#!pip install --upgrade numpy gensim Cython
#!pip install pandas numpy networkx matplotlib seaborn scikit-learn node2vec
!pip install umap-learn

    PyYAML (>=5.1.*)
            ~~~~~~^


   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------------------------------------- -- 2.6/2.8 MB 18.9 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   ------ --------------------------------- 5.0/30.3 MB 23.2 MB/s eta 0:00:02
   ------------- -------------------------- 10.5/30.3 MB 24.2 MB/s eta 0:00:01
   --------------------- ------------------ 16.3/30.3 MB 24.9 MB/s eta 0:00:01
   ---------------------------- ----------- 21.5/30.3 MB 24.7 MB/s eta 0:00:01
   ---------------------------------- ----- 26.5/30.3 MB 24.3 MB/s eta 0:00:01
   ---------------------------------------  30.1/30.3 MB 24.5 MB/s eta 0:00:01
   ---------------------------------------- 30.3/30.3 MB 20.7 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [llvmlite]
   ---------------------------------------- 0/4 [llvmlite]
   ----------------------------------

In [21]:
#!pip uninstall -y numpy scipy gensim Cython
#!pip install numpy==1.24.4 scipy==1.11.4 gensim==4.3.1 Cython
!pip install torch_geometric

    PyYAML (>=5.1.*)
            ~~~~~~^



   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.1 MB 764.3 kB/s eta 0:00:01
   ------------------ --------------------- 0.5/1.1 MB 764.3 kB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.1 MB 714.3 kB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.1 MB 719.5 kB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 707.9 kB/s eta 0:00:00


**Siin on GraphSAGE-iga tehtud katse luua võrgustikku**

In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

# Example: small graph with 4 nodes and edges
edge_index = torch.tensor([
    [0, 1, 1, 2, 3],
    [1, 0, 2, 1, 0]
], dtype=torch.long)

# Node features (4 nodes, 3 features each)
x = torch.randn((4, 3))

data = Data(x=x, edge_index=edge_index)

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = GraphSAGE(in_channels=3, hidden_channels=5, out_channels=2)

out = model(data.x, data.edge_index)
print(out)

C:\Users\molsi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[-0.0379,  0.1110],
        [ 0.2874, -0.2452],
        [ 0.3121, -0.0219],
        [-0.4075, -0.0450]], grad_fn=<AddBackward0>)


In [ ]:
import pandas as pd
import torch
from torch_geometric.data import Data
import os

# Ainult üks fail, et testida
file_path = "./data/ratings-00000.tsv"
ratings = pd.read_csv(file_path, sep='\t', low_memory=False)

# Vali ainult vajalikud veerud ja dropi tühjad
ratings = ratings[['noteId', 'raterParticipantId', 'helpful']]
ratings.dropna(subset=['noteId', 'raterParticipantId'], inplace=True)

unique_nodes = pd.Index(
    ratings['noteId'].astype(str).tolist() +
    ratings['raterParticipantId'].astype(str).tolist()
).unique()

node2idx = {node: idx for idx, node in enumerate(unique_nodes)}

# Loome edge_index tensorid
src = ratings['raterParticipantId'].astype(str).map(node2idx).values
dst = ratings['noteId'].astype(str).map(node2idx).values

edge_index = torch.tensor([src, dst], dtype=torch.long)

# Dummy node feature (kõik 1)
x = torch.ones((len(unique_nodes), 1), dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[1540527, 1], edge_index=[2, 13618878])


C:\Users\molsi\AppData\Local\Temp\ipykernel_14564\4115226570.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  edge_index = torch.tensor([src, dst], dtype=torch.long)


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import NeighborSampler

# Väike GraphSAGE mudel
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(in_channels=1, hidden_channels=64, out_channels=32).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

model.train()
for epoch in range(10): 
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = (out.norm(dim=1) ** 2).mean() 
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 8.9966
Epoch 2, Loss: 6.2593
Epoch 3, Loss: 3.8905
Epoch 4, Loss: 3.6471
Epoch 5, Loss: 3.5774
Epoch 6, Loss: 2.5202
Epoch 7, Loss: 1.3770
Epoch 8, Loss: 0.8565
Epoch 9, Loss: 0.9608
Epoch 10, Loss: 1.2818


In [4]:
model.eval()
embeddings = model(data.x, data.edge_index).detach().cpu().numpy()

The rest of the code above is now gone (didn't save before creating the network with large data - so it freezed and the code is gone)

**Below is how i made the current reults by using the TruncatedSVD, KMeans and umap**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from tqdm import tqdm

note_status = pd.read_csv("data/noteStatusHistory-00000.tsv", sep='\t')

print(note_status['mostRecentNonNMRStatus'].value_counts())

# Võta ainult approved notes
approved_notes = note_status[note_status['mostRecentNonNMRStatus'] == "CURRENTLY_RATED_HELPFUL"]['noteId'].unique()
print(f"Approved notes: {len(approved_notes)}")


ratings_files = [
    "data/ratings-00000.tsv",
    "data/ratings-00008.tsv"
]

# Vaid vajalikud veerud
usecols = ['noteId', 'raterParticipantId']

all_rows = []

for file in ratings_files:
    print(f"Reading {file} ...")
    chunksize = 10 ** 6  # loeme 1M rida korraga
    for chunk in pd.read_csv(file, sep='\t', usecols=usecols, chunksize=chunksize):
        # Filtreeri approved notes
        chunk = chunk[chunk['noteId'].isin(approved_notes)]
        all_rows.append(chunk)

ratings = pd.concat(all_rows, ignore_index=True)
print(f"Ratings shape after filtering: {ratings.shape}")


noteId_to_idx = {nid: i for i, nid in enumerate(ratings['noteId'].unique())}
raterId_to_idx = {rid: i for i, rid in enumerate(ratings['raterParticipantId'].unique())}

num_notes = len(noteId_to_idx)
num_raters = len(raterId_to_idx)

print(f"Number of unique notes: {num_notes}")
print(f"Number of unique raters: {num_raters}")

# Build CSR matrix
rows = ratings['noteId'].map(noteId_to_idx).values
cols = ratings['raterParticipantId'].map(raterId_to_idx).values
data = np.ones(len(ratings))

adj_matrix = csr_matrix((data, (rows, cols)), shape=(num_notes, num_raters))
print(f"Adjacency matrix shape: {adj_matrix.shape}")

# 4️ Embeddings

svd_dim = 32

svd_notes = TruncatedSVD(n_components=svd_dim, random_state=42)
print("adj_matrix:",adj_matrix)
print("svd_notes:",svd_notes)
notes_embeddings = svd_notes.fit_transform(adj_matrix)

svd_raters = TruncatedSVD(n_components=svd_dim, random_state=42)
raters_embeddings = svd_raters.fit_transform(adj_matrix.T)

print(f"Notes embeddings shape: {notes_embeddings.shape}")
print(f"Raters embeddings shape: {raters_embeddings.shape}")

# 5️ Clustering

num_note_clusters = 20
kmeans_notes = KMeans(n_clusters=num_note_clusters, random_state=42)
note_labels = kmeans_notes.fit_predict(notes_embeddings)

num_rater_clusters = 10
kmeans_raters = KMeans(n_clusters=num_rater_clusters, random_state=42)
rater_labels = kmeans_raters.fit_predict(raters_embeddings)

# 6️ Save results

note_idx_to_id = {v: k for k, v in noteId_to_idx.items()}
df_notes = pd.DataFrame(notes_embeddings, columns=[f"emb_{i}" for i in range(svd_dim)])
df_notes['noteId'] = df_notes.index.map(note_idx_to_id)
df_notes['cluster'] = note_labels

df_notes.to_csv("graph_embeddings_notes.csv", index=False)
print("Saved graph_embeddings_notes.csv")

# Raters dataframe
rater_idx_to_id = {v: k for k, v in raterId_to_idx.items()}
df_raters = pd.DataFrame(raters_embeddings, columns=[f"emb_{i}" for i in range(svd_dim)])
df_raters['raterParticipantId'] = df_raters.index.map(rater_idx_to_id)
df_raters['cluster'] = rater_labels

df_raters.to_csv("graph_embeddings_raters.csv", index=False)
print("Saved graph_embeddings_raters.csv")


C:\Users\molsi\AppData\Local\Temp\ipykernel_14564\281983218.py:10: DtypeWarning: Columns (10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  note_status = pd.read_csv("data/noteStatusHistory-00000.tsv", sep='\t')


CURRENTLY_RATED_HELPFUL        232367
CURRENTLY_RATED_NOT_HELPFUL    132457
Name: mostRecentNonNMRStatus, dtype: int64
Approved notes: 232367
Reading data/ratings-00000.tsv ...
Reading data/ratings-00008.tsv ...
Ratings shape after filtering: (6475758, 2)
Number of unique notes: 231274
Number of unique raters: 202897
Adjacency matrix shape: (231274, 202897)
adj_matrix:   (0, 0)	1.0
  (0, 31)	1.0
  (0, 189)	1.0
  (0, 776)	1.0
  (0, 1177)	1.0
  (0, 1196)	1.0
  (0, 1653)	1.0
  (0, 1756)	1.0
  (0, 1806)	1.0
  (0, 2068)	1.0
  (0, 2266)	1.0
  (0, 2282)	1.0
  (0, 2609)	1.0
  (0, 2626)	1.0
  (0, 2679)	1.0
  (0, 2774)	1.0
  (0, 3055)	1.0
  (0, 3180)	1.0
  (0, 3534)	1.0
  (0, 3812)	1.0
  (0, 3847)	1.0
  (0, 3862)	1.0
  (0, 3952)	1.0
  (0, 3972)	1.0
  (0, 4111)	1.0
  :	:
  (231249, 200970)	1.0
  (231250, 200970)	1.0
  (231251, 201234)	1.0
  (231252, 201580)	1.0
  (231253, 201709)	1.0
  (231254, 201725)	1.0
  (231255, 201734)	1.0
  (231256, 201781)	1.0
  (231257, 201831)	1.0
  (231258, 201834)	1.0

In [ ]:
import pandas as pd
import umap
import numpy as np
from tqdm import tqdm

# 1️ Loeme notes embeddings
df_notes = pd.read_csv("graph_embeddings_notes.csv")
print(f"Notes embeddings loaded: {df_notes.shape}")

# 2️ Loeme raters embeddings
df_raters = pd.read_csv("graph_embeddings_raters.csv")
print(f"Raters embeddings loaded: {df_raters.shape}")

# 3️ UMAP config
umap_model = umap.UMAP(n_neighbors=30, min_dist=0.1, n_components=2, random_state=42, verbose=True)

# 4️ Notes UMAP
print("Fitting UMAP on notes ...")
X_notes = df_notes.filter(like="emb_").values
notes_umap = umap_model.fit_transform(X_notes)

df_notes['umap_X'] = notes_umap[:, 0]
df_notes['umap_Y'] = notes_umap[:, 1]

# Save
df_notes.to_csv("graph_embeddings_notes_umap.csv", index=False)
print("Saved graph_embeddings_notes_umap.csv")

# 5️ Raters UMAP
print("Fitting UMAP on raters ...")
X_raters = df_raters.filter(like="emb_").values
raters_umap = umap_model.fit_transform(X_raters)

df_raters['umap_X'] = raters_umap[:, 0]
df_raters['umap_Y'] = raters_umap[:, 1]

# Save
df_raters.to_csv("graph_embeddings_raters_umap.csv", index=False)
print("Saved graph_embeddings_raters_umap.csv")


Notes embeddings loaded: (231274, 34)
Raters embeddings loaded: (202897, 34)
Fitting UMAP on notes ...
UMAP(n_jobs=1, n_neighbors=30, random_state=42, verbose=True)


C:\Users\molsi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Fri Jun  6 17:42:18 2025 Construct fuzzy simplicial set
Fri Jun  6 17:42:19 2025 Finding Nearest Neighbors
Fri Jun  6 17:42:19 2025 Building RP forest with 29 trees
Fri Jun  6 17:42:30 2025 NN descent for 18 iterations
	 1  /  18
	 2  /  18
	 3  /  18
	Stopping threshold met -- exiting after 3 iterations
Fri Jun  6 17:43:19 2025 Finished Nearest Neighbor Search
Fri Jun  6 17:43:23 2025 Construct embedding


Epochs completed:   1%|            2/200 [00:04]

	completed  0  /  200 epochs


...

	completed  160  /  200 epochs


Epochs completed:  91%| █████████  182/200 [03:50]

	completed  180  /  200 epochs


Epochs completed: 100%| ██████████ 200/200 [04:04]


Fri Jun  6 18:01:07 2025 Finished embedding
Saved graph_embeddings_raters_umap.csv


**Siin puhastama segadustekitava andmestiku ja loome networkid**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

df_notes = pd.read_csv("graph_embeddings_notes.csv")
df_raters = pd.read_csv("graph_embeddings_raters.csv")
ratings = pd.read_csv("data/ratings-00000.tsv", sep='\t', usecols=['noteId', 'raterParticipantId'])

# Lisa klastrid
ratings = ratings.merge(df_notes[['noteId', 'cluster']], on='noteId', how='inner')
ratings = ratings.merge(df_raters[['raterParticipantId', 'cluster']], on='raterParticipantId', how='inner', suffixes=('_note', '_rater'))

# ⛓️ Edge list: ühenduste tugevused rater-note klastri vahel
edge_df = ratings.groupby(['cluster_rater', 'cluster_note']).size().reset_index(name='weight')
edge_df['source'] = 'R_' + edge_df['cluster_rater'].astype(str)
edge_df['target'] = 'N_' + edge_df['cluster_note'].astype(str)

# Salvestame edge listi
edge_df[['source', 'target', 'weight']].to_csv("cluster_edges.csv", index=False)

# Node list: kõik sõlmed koos tüübiga ja suurusega
rater_cluster_sizes = df_raters['cluster'].value_counts().reset_index()
rater_cluster_sizes.columns = ['id', 'size']
rater_cluster_sizes['id'] = 'R_' + rater_cluster_sizes['id'].astype(str)
rater_cluster_sizes['type'] = 'rater'

note_cluster_sizes = df_notes['cluster'].value_counts().reset_index()
note_cluster_sizes.columns = ['id', 'size']
note_cluster_sizes['id'] = 'N_' + note_cluster_sizes['id'].astype(str)
note_cluster_sizes['type'] = 'note'

node_df = pd.concat([rater_cluster_sizes, note_cluster_sizes])
node_df.to_csv("cluster_nodes.csv", index=False)

In [33]:
one_note_per_cluster = df_notes.groupby('cluster')['noteId'].first().reset_index()
print(one_note_per_cluster)

    cluster               noteId
0         0  1871404259142459857
1         1  1865590698902503921
2         2  1679319580831883264
3         3  1779207915749916889
4         4  1844374392169812149
5         5  1733821630449668329
6         6  1762849587457622499
7         7  1702285661330722930
8         8  1677545815458594819
9         9  1771990281727963467
10       10  1782863374235099220
11       11  1800717246706196808
12       12  1727529309810655291
13       13  1673568883695923201
14       14  1714589572200304695
15       15  1776426205035819122
16       16  1818973938346815666
17       17  1727034787293274506
18       18  1818243351017341150
19       19  1710821689636934115


In [41]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOP%2B2AEAAAAAUa30N6XR7pwI1MySlBxCp5J2LEw%3DuQZOFyoSaBeDbKKDsj094lrFb9GEa8UMX7E1TjKphDA2XG1q94'

tweet_ids = [
    "1871404259142459857",
    "1865590698902503921",
    "1679319580831883264",
    "1779207915749916889",
    "1844374392169812149",
    "1733821630449668329",
    "1762849587457622499",
    "1702285661330722930",
    "1677545815458594819",
    "1771990281727963467",
    "1782863374235099220",
    "1800717246706196808",
    "1727529309810655291",
    "1673568883695923201",
    "1714589572200304695",
    "1776426205035819122",
    "1818973938346815666",
    "1727034787293274506",
    "1818243351017341150",
    "1710821689636934115"
]

def get_tweet_data(tweet_id, bearer_token):
    tweet_url = f"https://api.twitter.com/2/tweets/{tweet_id}"
    params = {
        "tweet.fields": "created_at,author_id,public_metrics,context_annotations",
        "expansions": "author_id"
    }
    headers = {
        "Authorization": f"Bearer {bearer_token}"
    }

    tweet_response = requests.get(tweet_url, headers=headers, params=params)

    if tweet_response.status_code == 200:
        tweet_data = tweet_response.json()
        tweet_text = tweet_data.get("data", {}).get("text", "Puudub tekst")
        author_id = tweet_data.get("data", {}).get("author_id", "Tundmatu")

        print("Säutsu tekst:")
        print(tweet_text)
        print("Säutsu link:")
        print( tweet_data.get("data", {}).get("url", "Puudub tekst"))
        #print("tweet_data:",tweet_data)
        print(f"\nAutor ID: {author_id}")
        return tweet_text
    else:
        print(f"Viga päringus tweetId {tweet_id}: {tweet_response.status_code}")
        return None
    
results = []

for tid in tweet_ids:
    data = get_tweet_data(tid, bearer_token)
    if data:
        # Võid siin töödelda või salvestada JSONi
        results.append(data)
    time.sleep(5)  # et mitte ületada rate limite (Twitter soovitab 1 päring sekundis)

# Näide: prindi esimese tweeti tekst
if results:
    print(results[0])

Viga päringus tweetId 1871404259142459857: 429
Viga päringus tweetId 1865590698902503921: 429


KeyboardInterrupt: 